In [1]:
#Imports

import os, sys
path_to_package = os.path.abspath(os.path.join('../..'))
if path_to_package not in sys.path:
    sys.path.append(path_to_package)


from src.io import read_phylip_file, read_phenotypes_file
from src.utils import *
from src.filters import *
from src.indexes import *
from src.anova import order_phenotypes_by_files_id

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


In [2]:
#Archivo de prueba: YBL088C

phenotypes_df = read_phenotypes_file("/home/khaldrem/code/sc_regmod/dataset/phenotypes/clean_phenotypes.csv")
data = read_phylip_file("/home/khaldrem/code/sc_regmod/dataset/anova/anova_at_least_one_phenotype/p_value_0_05/all/YAL001C.phylip")

In [3]:
df = pd.DataFrame()

cols_data = {}
for col in range(data.get_alignment_length()):
    key_name = "x" + str(col)
    cols_data[key_name] = list(data[:, col])
#print(cols_data[0])

df = pd.DataFrame.from_dict(cols_data, orient='columns')

phenotypes_df = order_phenotypes_by_files_id("/home/khaldrem/code/sc_regmod/dataset/anova/anova_at_least_one_phenotype/p_value_0_05/all/YAL001C.phylip", phenotypes_df)

In [4]:
phenotypes_df = phenotypes_df.reset_index()
phenotypes_df.drop('index', axis = 1, inplace= True)
phenotypes_df

,Standard,Haploide-Diploide,SM300-Efficiency,SM300-Rate,SM300-Lag,SM300-AUC,SM60-Efficiency,SM60-Rate,SM60-Lag,SM60-AUC,Ratio-Efficiency,Ratio-Rate,Ratio-Lag,Ratio-AUC,Ecological info,Standard_num
0,SACE_YAU,diploides-euploides,0.4674,0.02177,8.749,11.69,0.5235,0.01098,4.650,9.180,1.120026,0.504364,0.531489,0.785287,Unknown,0
1,SACE_YCF,diploides-euploides,0.6397,0.02024,7.291,13.72,0.5942,0.01256,5.822,10.020,0.928873,0.620553,0.798519,0.730321,Unknown,1
2,SACE_YAY,diploides-euploides,0.7460,0.02756,7.684,17.24,0.6401,0.01819,9.599,11.480,0.858043,0.660015,1.249219,0.665893,Domesticated (wine),2
3,SACE_YCR,diploides-euploides,0.7786,0.03354,6.953,20.02,0.5408,0.01095,7.606,8.117,0.694580,0.326476,1.093916,0.405445,Wild,3
4,SACE_YBG,diploides-euploides,1.0280,0.04192,9.167,23.65,0.7715,0.02017,6.011,15.230,0.750486,0.481155,0.655722,0.643975,Domesticated (wine),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,CDV,diploides-euploides,0.9027,0.03049,9.922,18.04,0.8780,0.01938,7.882,14.010,0.972638,0.635618,0.794396,0.776608,Domesticated (wine),942
943,CQI,diploides-euploides,0.8733,0.04427,6.694,24.36,0.6809,0.02250,5.623,15.900,0.779686,0.508245,0.840006,0.652709,Domesticated (non-wine),943
944,BPP,---,0.8654,0.02858,7.159,19.34,0.9155,0.02622,6.108,19.160,1.057892,0.917425,0.853192,0.990693,Domesticated (wine),944
945,CLS,diploides-euploides,1.2880,0.07294,12.900,29.76,0.9786,0.03971,12.800,19.220,0.759783,0.544420,0.992248,0.645833,Domesticated (non-wine),945


In [5]:
df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169
0,a,c,g,g,c,t,c,g,c,g,...,g,a,a,t,g,a,g,t,t,c
1,a,c,g,g,c,c,c,a,c,g,...,g,a,a,t,g,a,g,t,t,c
2,a,c,g,g,c,t,c,a,c,g,...,g,a,a,t,g,a,c,t,a,c
3,a,c,g,g,c,c,c,a,c,g,...,g,a,a,t,g,a,g,t,t,c
4,a,c,g,g,c,t,c,a,c,g,...,g,a,a,t,g,a,c,t,a,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,a,c,g,g,c,t,c,a,c,g,...,g,a,a,t,g,a,c,t,a,c
943,a,c,g,g,c,t,c,a,c,g,...,g,a,a,t,g,a,s,t,w,c
944,a,c,g,g,c,t,c,a,c,g,...,g,a,a,t,g,a,c,t,a,c
945,a,c,g,g,c,t,c,a,c,g,...,g,a,t,t,g,a,g,t,t,t


In [6]:
#df["SM300-Rate"] = phenotypes_df["SM300-Rate"].tolist()

df_keys = df.keys().tolist() + phenotypes_df.keys().tolist()
print(df_keys)
print(len(df_keys))

['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79', 'x80', 'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89', 'x90', 'x91', 'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100', 'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x107', 'x108', 'x109', 'x110', 'x111', 'x112', 'x113', 'x114', 'x115', 'x116', 'x117', 'x118', 'x119', 'x120', 'x121', 'x122', 'x123', 'x124', 'x125', 'x126', 'x127', 'x128', 'x129', 'x130', 'x131', 'x132', 'x133', 'x134', 'x135', 'x136', 'x137', 'x138

In [7]:

df = pd.concat([df, phenotypes_df], axis='columns', ignore_index=True)
df

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,a,c,g,g,c,t,c,g,c,g,...,0.5235,0.01098,4.650,9.180,1.120026,0.504364,0.531489,0.785287,Unknown,0
1,a,c,g,g,c,c,c,a,c,g,...,0.5942,0.01256,5.822,10.020,0.928873,0.620553,0.798519,0.730321,Unknown,1
2,a,c,g,g,c,t,c,a,c,g,...,0.6401,0.01819,9.599,11.480,0.858043,0.660015,1.249219,0.665893,Domesticated (wine),2
3,a,c,g,g,c,c,c,a,c,g,...,0.5408,0.01095,7.606,8.117,0.694580,0.326476,1.093916,0.405445,Wild,3
4,a,c,g,g,c,t,c,a,c,g,...,0.7715,0.02017,6.011,15.230,0.750486,0.481155,0.655722,0.643975,Domesticated (wine),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,a,c,g,g,c,t,c,a,c,g,...,0.8780,0.01938,7.882,14.010,0.972638,0.635618,0.794396,0.776608,Domesticated (wine),942
943,a,c,g,g,c,t,c,a,c,g,...,0.6809,0.02250,5.623,15.900,0.779686,0.508245,0.840006,0.652709,Domesticated (non-wine),943
944,a,c,g,g,c,t,c,a,c,g,...,0.9155,0.02622,6.108,19.160,1.057892,0.917425,0.853192,0.990693,Domesticated (wine),944
945,a,c,g,g,c,t,c,a,c,g,...,0.9786,0.03971,12.800,19.220,0.759783,0.544420,0.992248,0.645833,Domesticated (non-wine),945


In [8]:
df.columns = df_keys

In [9]:
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,SM60-Efficiency,SM60-Rate,SM60-Lag,SM60-AUC,Ratio-Efficiency,Ratio-Rate,Ratio-Lag,Ratio-AUC,Ecological info,Standard_num
0,a,c,g,g,c,t,c,g,c,g,...,0.5235,0.01098,4.650,9.180,1.120026,0.504364,0.531489,0.785287,Unknown,0
1,a,c,g,g,c,c,c,a,c,g,...,0.5942,0.01256,5.822,10.020,0.928873,0.620553,0.798519,0.730321,Unknown,1
2,a,c,g,g,c,t,c,a,c,g,...,0.6401,0.01819,9.599,11.480,0.858043,0.660015,1.249219,0.665893,Domesticated (wine),2
3,a,c,g,g,c,c,c,a,c,g,...,0.5408,0.01095,7.606,8.117,0.694580,0.326476,1.093916,0.405445,Wild,3
4,a,c,g,g,c,t,c,a,c,g,...,0.7715,0.02017,6.011,15.230,0.750486,0.481155,0.655722,0.643975,Domesticated (wine),4


In [13]:
for k in df.keys().tolist():
    print(k)

x0
x1
x2
x3
x4
x5
x6
x7
x8
x9
x10
x11
x12
x13
x14
x15
x16
x17
x18
x19
x20
x21
x22
x23
x24
x25
x26
x27
x28
x29
x30
x31
x32
x33
x34
x35
x36
x37
x38
x39
x40
x41
x42
x43
x44
x45
x46
x47
x48
x49
x50
x51
x52
x53
x54
x55
x56
x57
x58
x59
x60
x61
x62
x63
x64
x65
x66
x67
x68
x69
x70
x71
x72
x73
x74
x75
x76
x77
x78
x79
x80
x81
x82
x83
x84
x85
x86
x87
x88
x89
x90
x91
x92
x93
x94
x95
x96
x97
x98
x99
x100
x101
x102
x103
x104
x105
x106
x107
x108
x109
x110
x111
x112
x113
x114
x115
x116
x117
x118
x119
x120
x121
x122
x123
x124
x125
x126
x127
x128
x129
x130
x131
x132
x133
x134
x135
x136
x137
x138
x139
x140
x141
x142
x143
x144
x145
x146
x147
x148
x149
x150
x151
x152
x153
x154
x155
x156
x157
x158
x159
x160
x161
x162
x163
x164
x165
x166
x167
x168
x169
Standard
Haploide-Diploide
SM300-Efficiency
SM300-Rate
SM300-Lag
SM300-AUC
SM60-Efficiency
SM60-Rate
SM60-Lag
SM60-AUC
Ratio-Efficiency
Ratio-Rate
Ratio-Lag
Ratio-AUC
Ecological info
Standard_num


In [ ]:
#Transform data into a DataFrame

df = pd.DataFrame()

#Id row
list_ids = []
for row in data:
    list_ids.append(row.id)

df.insert(0, "ids", list_ids)

#Data rows
index = 1
for col in range(data.get_alignment_length()):
    col_name = "x" + str(index)
    df.insert(index, col_name, list(data[:, col]))
    index += 1


#Y column
phenotypes_ordered = []
SELECTED_PHENOTYPE = "SM300-Efficiency"

for ind in df.index:
    value = float(phenotypes.loc[phenotypes['Standard'] == df['ids'][ind]][SELECTED_PHENOTYPE])
    phenotypes_ordered.append(value)

df.insert(index, "y", phenotypes_ordered)

df.head()

In [ ]:
#Drop ids col
#One hot encoding
df_ohe = df.drop(columns=["ids"])
df_ohe = pd.get_dummies(df_ohe)

df_ohe.head()

In [ ]:
#Train Test Data!

#Labels
labels = np.array(df_ohe["y"])

#Features
df_ohe_wo_y = df_ohe.drop(columns=["y"])
features_list = list(df_ohe_wo_y.columns)

features = np.array(df_ohe_wo_y)

#Train & test set
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state=42)

In [ ]:
# Model

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)

#Train
rf.fit(train_features, train_labels)



In [ ]:
#Predictions

predictions = rf.predict(test_features)

#Error
errors = abs(predictions - test_labels)

print(f"Mean Abs Error: {round(np.mean(errors), 2)}")

In [ ]:
#MAPE
mape = 100 * (errors/test_labels)

accuracy = 100 - np.mean(mape)
print(f"Accuracy: {round(accuracy, 2)}%")

In [ ]:
#Feature importance
importances = list(rf.feature_importances_)

feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_list, importances)]

#sort
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse=True)

[print("Var: {:20} Importance: {}".format(*pair)) for pair in feature_importances];